# Рекомендация тарифов для клиентов оператора связи "Мегалайн"
## Постановка задачи

Согласно данным оператора мобильной связи «Мегалайн», многие клиенты пользуются архивными тарифами. Оператор хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

На основании данных о поведении клиентов, которые уже перешли на эти тарифы, нужно построить модель для задачи классификации, которая выберет подходящий тариф для каждого нового клиента. Данные готовы к анализу, предобработка данных не нужна.

Поставлена задача - на основании данных построить и обучить модель с максимально большим значением accuracy, доля правильных ответов в предсказаниях должна быть по крайней мере 0.75. Затем нужно проверить accuracy на тестовой выборке.

Выбрав лучшую из обученных моделей, нужно провести проверку это модели на вменяемость. В случае успешного прохождения проверки модель можно рекомендовать для использования заказчиком.

## Импорт данных

Импортируем датасет в переменную `df`:

In [68]:
from pathlib import Path

import pandas as pd
pd.set_option('display.float_format', '{:,.4f}'.format)

import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [26]:
my_path = Path('/home/klarazetkin/Documents/yandex/module_2/project_2')
if my_path.is_dir():
    df = pd.read_csv('/home/klarazetkin/Documents/yandex/module_2/project_2/users_behavior.csv')
else:
    df = pd.read_csv('/datasets/users_behavior.csv')

## Проверка и предобработка данных

Ознакомимся с данными, проверим, что предобработка действительно не нужна:

In [27]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0000,311.9000,83.0000,"19,915.4200",0
1,85.0000,516.7500,56.0000,"22,696.9600",0
2,77.0000,467.6600,86.0000,"21,060.4500",0
3,106.0000,745.5300,81.0000,"8,437.3900",1
4,66.0000,418.7400,1.0000,"14,502.7500",0


In [28]:
df.shape

(3214, 5)

In [29]:
df.isna().count()

calls       3214
minutes     3214
messages    3214
mb_used     3214
is_ultra    3214
dtype: int64

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Преобразуем данные в колонке `calls` к типу `int`, данные в колонке `is_ultra` к типу `boolean`:

In [31]:
df['calls'] = df['calls'].astype('int')
df['is_ultra'] = df['is_ultra'].astype('bool')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null int64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null bool
dtypes: bool(1), float64(3), int64(1)
memory usage: 103.7 KB


Согласно задания, тщательную проверку и предобработку данных делать не надо, поэтому ограничимся сделанным.

Отдельно учтем, что в исходном датафрейме пользователей тарифа "Ультра" 31 %, а пользователей тарифа "Смарт" 69%:

In [33]:
round(df['is_ultra'].sum() / len(df) * 100)

31

## Выделение обучающей, валидационной и тестовой выборок

Разделим имеющийся датасет на три части: обучающую (60%), валидационную (20%) и тестовую (20%) выборки. Укажем параметр `random_state=12345` для обеспечения повторяемости результата.

In [34]:
df_train, df_valid_and_test = train_test_split(df, test_size=0.4, random_state=12345)

In [35]:
df_valid, df_test = train_test_split(df_valid_and_test, test_size=0.5, random_state=12345)

In [36]:
# проверим размеры выборок
print(len(df_train))
print(len(df_valid))
print(len(df_test))

1928
643
643


## Определение целевого признака и других признаков

Целевой признак - подходящий тариф, ему соответствует колонка `is_ultra`. Все остальные признаки попадут в `features`.

In [37]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

Определим целевой и обучающие признаки для тестовой выборки:

In [38]:
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

## Построение моделей классификации разных типов и выбор лучшей модели для предсказания целевого признака
Построим три модели: дерево решений, случайный лес и  логистическую регрессию, подберем для них оптимальные параметры, затем сравним лучшие показатели моделей между собой.

### Решающее дерево
#### Подбор гиперпараметров и выбор лучшей модели

Построим модель решающего дерева. Поэкспериментируем с параметром `depth`. Лучший показатель `accuracy = 0.7853810264385692` у модели с глубиной `depth = 3`.

In [39]:
best_model = None
best_accuracy = 0

for depth in range(1, 10):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print("max_depth =", depth, ": ", end='  ')
    print(accuracy_score(target_valid, predictions_valid)) 
    if accuracy_score(target_valid, predictions_valid) > best_accuracy:
        best_accuracy = accuracy_score(target_valid, predictions_valid)
        best_model = model

max_depth = 1 :   0.7542768273716952
max_depth = 2 :   0.7822706065318819
max_depth = 3 :   0.7853810264385692
max_depth = 4 :   0.7791601866251944
max_depth = 5 :   0.7791601866251944
max_depth = 6 :   0.7838258164852255
max_depth = 7 :   0.7822706065318819
max_depth = 8 :   0.7791601866251944
max_depth = 9 :   0.7822706065318819


Поэкспериментируем с другими показателями  - минимальным количеством объектов в узле `min_samples_split` и минимальным количеством объектов в итоговом листе - `min_samples_leaf`.


In [40]:
best_tree_model = None
best_tree_accuracy = 0

for depth in range(1, 10):
    for min_split in range(2, 113, 10):
        for min_leaf in range(2, 113, 10):
            model = DecisionTreeClassifier(
                max_depth=depth, min_samples_split=min_split, min_samples_leaf=min_leaf, random_state=12345)
            model.fit(features_train, target_train)
            predictions_valid = model.predict(features_valid)
            print("max_depth =", depth, ": ", end='  ')
            print("min_samples_split =", min_split, end='  ')
            print("min_samples_leaf =", min_leaf, end='  ')
            print(accuracy_score(target_valid, predictions_valid)) 
            if accuracy_score(target_valid, predictions_valid) > best_tree_accuracy:
                best_tree_accuracy = accuracy_score(target_valid, predictions_valid)
                best_tree_model = model
                best_depth = depth
                best_min_samples_split = min_split
                best_min_samples_leaf = min_leaf
                
print("Accuracy наилучшей модели дерева на валидационной выборке:", best_tree_accuracy)

max_depth = 1 :   min_samples_split = 2  min_samples_leaf = 2  0.7542768273716952
max_depth = 1 :   min_samples_split = 2  min_samples_leaf = 12  0.7542768273716952
max_depth = 1 :   min_samples_split = 2  min_samples_leaf = 22  0.7542768273716952
max_depth = 1 :   min_samples_split = 2  min_samples_leaf = 32  0.7542768273716952
max_depth = 1 :   min_samples_split = 2  min_samples_leaf = 42  0.7542768273716952
max_depth = 1 :   min_samples_split = 2  min_samples_leaf = 52  0.7542768273716952
max_depth = 1 :   min_samples_split = 2  min_samples_leaf = 62  0.7542768273716952
max_depth = 1 :   min_samples_split = 2  min_samples_leaf = 72  0.7542768273716952
max_depth = 1 :   min_samples_split = 2  min_samples_leaf = 82  0.7542768273716952
max_depth = 1 :   min_samples_split = 2  min_samples_leaf = 92  0.7542768273716952
max_depth = 1 :   min_samples_split = 2  min_samples_leaf = 102  0.7542768273716952
max_depth = 1 :   min_samples_split = 2  min_samples_leaf = 112  0.7542768273716952
max

В результате подбора гиперпараметров `min_samples_leaf` и `min_samples_split` удалось получить модель с более высоким значением `accuracy = 0.7884914463452566` против `accuracy = 0.7853810264385692` у модели, где подобрана только глубина дерева (`depth = 3`). У новой лучшей модели дерева глубина - `depth = 6`.

**Вывод:**

Лучше всего себя показала модель с глубиной 6, минимальным количеством объектов в узле 82 и минимальным листом в 2 объекта. Однако есть риск, что модель переобучена. Поскольку при глубине меньше четырёх решающее дерево часто недообучается, а при глубине больше — переобучается.

Выведем данные по лучшей модели - модели с максимальной `accuracy`. 

In [41]:
print(best_tree_model)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=82,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')


#### Проверка дерева решений на тестовой выборке

Проверим лучшую модель дерева решений на тестовой выборке. Получено значение `accuracy = 0.7947122861586314`, что даже выше, чем `accuracy = 0.7884914463452566` на валидационной выборке.

In [42]:
predictions_test = best_tree_model.predict(features_test)
tree_accuracy_test = accuracy_score(target_test, predictions_test)
print("Accuracy наилучшей модели дерева на тестовой выборке:", tree_accuracy_test)

Accuracy наилучшей модели дерева на тестовой выборке: 0.7947122861586314


### Случайный лес
#### Подбор гиперпараметров и выбор лучшей модели случайного леса

Построим и обучим модель случайного леса, попробуем подобрать гиперпараметры - количество деревьев-оценщиков.

In [43]:
best_forest_model = None
best_forest_result = 0
for est in range(1, 211, 5):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    print("estimators =", est, ": ", end='  ')
    print(result)
    if result > best_forest_result:
        best_forest_model = model 
        best_forest_result = result
print("Accuracy наилучшей модели леса на валидационной выборке:", best_forest_result)

estimators = 1 :   0.7107309486780715
estimators = 6 :   0.7807153965785381
estimators = 11 :   0.7838258164852255
estimators = 16 :   0.7869362363919129
estimators = 21 :   0.7931570762052877
estimators = 26 :   0.7853810264385692
estimators = 31 :   0.7822706065318819
estimators = 36 :   0.7807153965785381
estimators = 41 :   0.7869362363919129
estimators = 46 :   0.7884914463452566
estimators = 51 :   0.7916018662519441
estimators = 56 :   0.7853810264385692
estimators = 61 :   0.7807153965785381
estimators = 66 :   0.7822706065318819
estimators = 71 :   0.7807153965785381
estimators = 76 :   0.7838258164852255
estimators = 81 :   0.7807153965785381
estimators = 86 :   0.7853810264385692
estimators = 91 :   0.7838258164852255
estimators = 96 :   0.7838258164852255
estimators = 101 :   0.7853810264385692
estimators = 106 :   0.7869362363919129
estimators = 111 :   0.7822706065318819
estimators = 116 :   0.7853810264385692
estimators = 121 :   0.7869362363919129
estimators = 126 :   0

In [370]:
print(best_forest_model)

RandomForestClassifier(n_estimators=21, random_state=12345)


Попробуем подобрать другие параметры для деревьев леса: максимальную глубину дерева, минимальное количество объектов в узле и минимальное количество объектов в листе.
    
Лучше всего себя показала модель RandomForestClassifier(max_depth=6, min_samples_leaf=2, n_estimators=36, random_state=12345). Сохраним эту модель в переменную `best_forest_model_2`.

In [11]:
best_forest_model_2 = None
best_forest_result_2 = 0
for est in range(1, 140, 5):
    for depth in range(1, 8):
        for min_split in range(2, 21):
            for min_leaf in range(2, 21, 2):
                model = RandomForestClassifier(
                    random_state=12345, 
                    n_estimators=est, 
                    max_depth=depth, 
                    min_samples_split=min_split, 
                    min_samples_leaf=min_leaf)
                model.fit(features_train, target_train)
                result = model.score(features_valid, target_valid)
                print("estimators =", est, ": ", end='  ')
                print("max_depth =", depth, ": ", end='  ')
                print("min_samples_split =", min_split, end='  ')
                print("min_samples_leaf =", min_leaf, end='  ')
                print(result)
                if result > best_forest_result_2:
                    best_forest_model_2 = model 
                    best_forest_result_2 = result
                
print("Accuracy наилучшей модели леса на валидационной выборке:", best_forest_result_2)
                

NameError: name 'features_train' is not defined

In [44]:
best_forest_model_2 = RandomForestClassifier(
                        random_state=12345, 
                        n_estimators=36, 
                        max_depth=6, 
                        min_samples_leaf=2)
best_forest_model_2.fit(features_train, target_train)
result = best_forest_model_2.score(features_valid, target_valid)
print("Accuracy кастомизированной модели случайного леса на валидационной выборке:", result)

Accuracy кастомизированной модели случайного леса на валидационной выборке: 0.8087091757387247


In [45]:
print(best_forest_model_2)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=6, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=36,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)


**Вывод:** 

Получено две лучших модели леса. 

Первая - `best_forest_model` имеет 21 дерево-оценщик и `accuracy = 0.7931570762052877`. 

Вторая - `best_forest_model_2` имеет 36 деревьев-оценщиков, максимальную глубину - 6, минимальное количество объектов в листе - 2. Для этой модели `accuracy = 0.8087091757387247`. 

Однако этот результат вполне может оказаться случайностью, т.к. обычно точность предсказаний растет с количеством деревьев-оценщиков, и маловероятно, что лес из 21 или 36 деревьев будет предсказывать точнее, чем лес с 100+ деревьями. Проверим это предположение на тестовой выборке.

#### Проверка случайного леса на тестовой выборке

Проверим первую лучшую модель случайного леса (c 21 деревом-оценщиком) на тестовой выборке. Получена `accuracy = 0.776049766718507`, что ниже значения для валидационной выборки `accuracy = 0.7931570762052877`.

In [46]:
forest_accuracy_test = best_forest_model.score(features_test, target_test)
print("Accuracy первой лучшей модели случайного леса на тестовой выборке:", forest_accuracy_test)

Accuracy первой лучшей модели случайного леса на тестовой выборке: 0.776049766718507


Попробуем еще один вариант - модель с 141 деревом-оценщиком. Возможно, результат на тестовой выборке будет лучше, чем у модели с 21 деревом.

In [47]:
forest_model_141_est = RandomForestClassifier(random_state=12345, n_estimators=141)
forest_model_141_est.fit(features_train, target_train)
result = forest_model_141_est.score(features_valid, target_valid)
print("Accuracy модели случайного леса с 141 деревом на валидационной выборке:", result)

Accuracy модели случайного леса с 141 деревом на валидационной выборке: 0.7900466562986003


In [48]:
forest_accuracy_test = forest_model_141_est.score(features_test, target_test)
print("Accuracy модели случайного леса c 141 деревом на тестовой выборке:", forest_accuracy_test)

Accuracy модели случайного леса c 141 деревом на тестовой выборке: 0.7869362363919129


`Accuracy` получилась выше, чем для леса с 21 деревом.

Попробуем еще один вариант с 206 деревьями - на валидационной выборке он показал такой же результат, как и вариант с 141 деревьями, но на тестовой что-то может измениться. Действительно, вариант с 206 деревьями-оценщиками показал более высокое значение `accuracy = 0.7884914463452566` на тестовой выборке. Это значение совпало со значением `accuracy` для леса с 21 деревом. 

In [53]:
forest_model_206_est = RandomForestClassifier(random_state=12345, n_estimators=206)
forest_model_206_est.fit(features_train, target_train)
result = forest_model_206_est.score(features_valid, target_valid)
print("Accuracy модели случайного леса с 206 деревьями на валидационной выборке:", result)

Accuracy модели случайного леса с 206 деревьями на валидационной выборке: 0.7900466562986003


In [54]:
forest_accuracy_test = forest_model_206_est.score(features_test, target_test)
print("Accuracy модели случайного леса c 206 деревьями на тестовой выборке:", forest_accuracy_test)

Accuracy модели случайного леса c 206 деревьями на тестовой выборке: 0.7884914463452566


Проверим вторую лучшую модель случайного леса с подобранными гиперпараметрами - `best_forest_model_2`:

In [55]:
forest_accuracy_test_2 = best_forest_model_2.score(features_test, target_test)
print("Accuracy кастомизированной наилучшей модели случайного леса на тестовой выборке:", forest_accuracy_test_2)

Accuracy кастомизированной наилучшей модели случайного леса на тестовой выборке: 0.7962674961119751


Для второй лучшей модели случайного леса значение `accuracy = 0.7962674961119751` выше, чем все ранее полученные значения `accuracy` для моделей случайного леса. 

Признаем эту модель самой удачной из моделей случайного леса и сохраним ее в переменную `best_forest_model`, а соответствующие ей значения `accuracy` на валидационной и тестовой выборках сохраним в переменные `best_forest_result` и `forest_accuracy_test`. 

In [56]:
best_forest_model = best_forest_model_2
best_forest_result = best_forest_model_2.score(features_valid, target_valid)
forest_accuracy_test = forest_accuracy_test_2

### Логистическая регрессия
#### Подбор гиперпараметров и выбор лучшей модели

Построим модель логистической регрессии. Попробуем подобрать гиперпараметры. Модель с гиперпараметрами по умолчанию дает низкую `accuracy = 0.7107309486780715`

In [71]:
regression_model = LogisticRegression(random_state=67890)
regression_model.fit(features_train, target_train)
result = regression_model.score(features_valid, target_valid)
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Попробуем указать гиперпараметр `solver='liblinear'` - данный алгоритм оптимизации подходит для небольших датасетов. `Accuracy` модели повысилась и стала `0.7589424572317263`. Этот гиперпараметр ограничивает выбор параметра `multi_class='ovr'`.

In [73]:
regression_model = LogisticRegression(random_state=12345, solver='liblinear')
regression_model.fit(features_train, target_train)
result = regression_model.score(features_valid, target_valid)
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


Попробуем изменить гиперпараметр толерантности. 

По умолчанию `tol=1e-4`. По результатам проверки, при этом значении получается самая высокая `accuracy = 0.7589424572317263`. При установке `tol=1e-3` и больше значение accuracy сильно падает, при установке `tol=1e-5` и меньше значение accuracy тоже падает, хотя и менее сильно.

In [74]:
regression_model = LogisticRegression(random_state=12345, solver='liblinear', tol=1e-4)
regression_model.fit(features_train, target_train)
result = regression_model.score(features_valid, target_valid)
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


In [75]:
best_regression_model = None
best_regression_result = 0

for degree in range(-10, -1):
    regression_model = LogisticRegression(random_state=12345, solver='liblinear', tol=10**degree)
    regression_model.fit(features_train, target_train)
    result = regression_model.score(features_valid, target_valid)
    print("tol =", 10**degree, ": ", end='  ')
    print(result)
    if result > best_regression_result:
        best_regression_model = regression_model 
        best_regression_result = result
print("Accuracy наилучшей модели логистической регрессии на валидационной выборке:", best_regression_result)

tol = 1e-10 :   0.7573872472783826
tol = 1e-09 :   0.7573872472783826
tol = 1e-08 :   0.7573872472783826
tol = 1e-07 :   0.7573872472783826
tol = 1e-06 :   0.7573872472783826
tol = 1e-05 :   0.7573872472783826
tol = 0.0001 :   0.7589424572317263
tol = 0.001 :   0.7076205287713841
tol = 0.01 :   0.7060653188180405
Accuracy наилучшей модели логистической регрессии на валидационной выборке: 0.7589424572317263


Изменение гиперпараметра количества итераций ничего не дало. По дефолту `max_iter=100`. При `max_iter <= 20` accuracy падает. Все значения от 30 и выше дают одну и ту же accuracy; при значениях `max_iter <= 30` сыплются ворнинги:

In [76]:
regression_model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=20)
regression_model.fit(features_train, target_train)
result = regression_model.score(features_valid, target_valid)
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7091757387247278


/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [77]:
print(best_regression_model)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)


**Вывод:**

Лучше всего себя показала модель регрессии с гиперпараметром `solver='liblinear'` и остальными параметрами по умолчанию: `multi_class='ovr', tol=1e-4, max_iter=100`. При заданных гиперпараметрах модель показывает максимальную `accuracy = 0.7589424572317263`.

#### Проверка логистической регрессии на тестовой выборке

Проверим лучшую модель регрессии на тестовой выборке. Полученный результат `accuracy = 0.7402799377916018`, что значительно ниже, чем на валидационной выборке с `accuracy = 0.7589424572317263`.

In [78]:
regression_accuracy_test = best_regression_model.score(features_test, target_test)
print("Accuracy наилучшей модели логистической регрессии на тестовой выборке:", regression_accuracy_test)

Accuracy наилучшей модели логистической регрессии на тестовой выборке: 0.7402799377916018


### Сравнение трех моделей

Сравним лучшие результаты по трем моделям. На тестовой выборке лучше всего показал себя случайный лес с `accuracy = 0.7963`. Результат у решающего дерева чуть ниже - `accuracy = 0.7947`.

In [79]:
output_table = pd.DataFrame(
    [['Decision Tree', best_tree_accuracy, tree_accuracy_test],
    ['Random Forest', best_forest_result, forest_accuracy_test],
    ['Logistic Regression', best_regression_result, regression_accuracy_test]],
    columns=['model', 'valid_accuracy', 'test_accuracy']
)
display(output_table)

,model,valid_accuracy,test_accuracy
0,Decision Tree,0.7885,0.7947
1,Random Forest,0.8087,0.7963
2,Logistic Regression,0.7589,0.7403


### Проверка на вменяемость

Проверим на вменяемость лучшую из наших моделей - модель случайного леса. Для этого построим простейшую модель, которая всегда будет предсказывать целевой признак `is_ultra = False`. Назовем ее `dummy_predictor`. 
После этого сравним accuracy для `best_forest_model` и `dummy_predictor`.

In [80]:
def dummy_predictor(array_length):
    return np.asarray(([False] * array_length), dtype=bool)


In [81]:
# проверим accuracy для dummy_predictor
dummy_predictions_test = dummy_predictor(len(features_test))
# print(dummy_predictions_test)

dummy_accuracy_test = accuracy_score(target_test, dummy_predictions_test)

print("Accuracy для dummy_predictor на тестовой выборке:", dummy_accuracy_test)

Accuracy для dummy_predictor на тестовой выборке: 0.6842923794712286


In [82]:
print("Accuracy лучшей модели случайного леса на тестовой выборке:", forest_accuracy_test)

Accuracy лучшей модели случайного леса на тестовой выборке: 0.7962674961119751


**Вывод:**

Лучшая из обученных моделей - модель случайного леса - прошла проверку на вменяемость, показав на тестовой выборке результат `accuracy = 0.7962674961119751`, против  `accuracy = 0.6842923794712286` простейшей модели, которая всегда предсказывает одно и то же.

## Общий вывод по проекту



Для решения задачи рекомендации тарифного плана были построены и обучены модели трех типов: решающее дерево, случайный лес и логистическая регрессия. Был произведен подбор гиперпараметров моделей и выбрана лучшая модель для каждого типа.

Параметр `accuracy` при работе на валидационной и тестовой выборках для трех лучших моделей приведен в таблице:

In [396]:
display(output_table)

,model,valid_accuracy,test_accuracy
0,Decision Tree,0.7885,0.7947
1,Random Forest,0.8087,0.7963
2,Logistic Regression,0.7589,0.7403


Лучше всего себя показала модель решающего леса с параметрами: `max_depth=6`, `min_samples_leaf=2`, `n_estimators=36`.

Эта модель прошла проверку на вменяемость, показав на тестовой выборке результат `accuracy = 0.7962674961119751`, против `accuracy = 0.6842923794712286` простейшей модели, которая всегда предсказывает одно и то же (`is_ultra = False`). 

Модель рекомендуется дообучить и начать использовать в бизнесе.